# Deploy a Python function
Code adapted from <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/1eddc77b3a4340d68f762625d40b64f9?context=cpdaas&audience=wdp">Use Python to recognize hand-written digits</a>

In this notebook, we create and deploy a Python function that calls an already deployed model.<br/>
We could have retrieve the `space_id` and `model_deployment_id` programmatically but it is 
outside the scope of this exercise.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'us-south'

space_id= "PASTE space GUID VALUE HERE"
model_deployment_id = "PASTE THE 'attrition challenger - spark deployment' guid VALUE HERE"

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

## Install the IBM Watson Machine Learning library
We redirect the output of the execution to a local file to limit what is displayed.

If you really want to see the output, create a new cell and execute the following command:
```
!cat wml_lib_install.out
```

In [ ]:
!pip install -U ibm-watson-machine-learning >wml_lib_install.out 2>&1

## Create a wml client connection
This is a client connection to our Watson Machine Learning (WML)

In [ ]:
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(wml_credentials)

## Create a function that calls a model
We are calling a previously deployed model named `attrition challenger - spark deployment`

We see two functions in `attrition_function`. The `score` function is the execution
point in the REST API call. Since both `score` and `reformat_output` are under 
`attrition_function`, `reformat_output` is accessible to `score` when it executes.

In [ ]:
params = {
    "wml_credentials": wml_credentials, 
    "space_id": space_id, 
    "model_deployment_id": model_deployment_id
}

def attrition_function( parms=params ):
    def reformat_output(result) :
        predictions = []
        for pred in result['predictions'] :
            fields=["probability", "prediction"]
            prob_index = pred["fields"].index("probability")
            pred_index = pred["fields"].index("prediction")
            values = [[item[prob_index][int(item[pred_index])], item[pred_index]] for item in pred["values"]]
            predictions.append( {"fields": fields, "values": values})

        result2 = {
            "predictions": predictions
        }
        return result2
    
    def score(payload):   
        try:                 
            from ibm_watson_machine_learning import APIClient
            client       = APIClient(parms["wml_credentials"])
            client.set.default_space(parms["space_id"])
            model_result = client.deployments.score(parms["model_deployment_id"], payload)
            return reformat_output(model_result)
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
            #return {"error" : repr(e)}
    return score

In [ ]:
# Test locally before deploying. This is useful for debugging
payload = {
    "input_data": [{
        "fields": ["POSITION_CODE","DEPARTMENT_CODE","DAYS_WITH_COMPANY","COMMUTE_TIME",
                   "AGE_BEGIN_PERIOD","GENDER_CODE","PERIOD_TOTAL_DAYS","STARTING_SALARY",
                   "ENDING_SALARY","NB_INCREASES","BONUS","NB_BONUS","VACATION_DAYS_TAKEN",
                   "SICK_DAYS_TAKEN","PROMOTIONS","NB_MANAGERS","DAYS_IN_POSITION",
                   "DAYS_SINCE_LAST_RAISE","RANKING_CODE","OVERTIME","DBLOVERTIME","TRAVEL"],
        "values": [[4500.0,360.0,910.0,22.0,56.0,1.0,330.0,87307.69,94615.38,1.0,
                    5676.9228,1.0,14.0,9.0,0.0,1.0,910.0,180.0,4.0,0.0,0.0,0.0]]
                    }]
}
func_result = attrition_function()(payload)
print(func_result)

## Deploy the function
We use the capabilities from the `ibm-watson-machine-learning` to deploy the function.

See the <a href="https://cloud.ibm.com/apidocs/machine-learning-cp?code=python">Watson Machine Learning</a> 
documentation for more details.

In [ ]:
client.set.default_space(space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'attrition function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=attrition_function)

In [ ]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "attrition function",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)

## Test the deployed function
Note that we used informatin we got while deploying the function.<br/>
If we were to come directly to this section, we would have to make 
additional calls to retrieve this information.

In [ ]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))

### Get the data to score
We could write the content to a local file with:
```
r = requests.get(url)
file_name = url.split('/')[-1]
with open(file_name, "wb") as file:
        file.write(r.content)
```
Then we'd have to read the local file

In [ ]:
# Same file we used in an earlier exercise
url = "https://raw.githubusercontent.com/CloudPak-Outcomes/Outcomes-Projects/main/TrustedAI-L3-Tech-Lab/records_to_score.json"
r = requests.get(url)
payload = json.loads(r.content.decode("utf-8"))

### Execute the function and see the result

In [ ]:
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)